<a href="https://colab.research.google.com/github/jiuyuanx/11663-LLM/blob/main/Vertex_AI_PaLM2_fine_tuning_for_cypher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

## **Dataset**

In [ ]:
df1=pd.read_csv("/content/drive/MyDrive/cypher/easy_cypher.csv", header=None, names=['question', 'cypher'])
df1.head(5)

,question,cypher
0,How many papers are in the database?,MATCH (p:paper) RETURN count(p) AS num_papers
1,Count the number of authors.,MATCH (a:Author) RETURN count(a) AS authorCount
2,How many conferences are listed?,MATCH (c:conference) RETURN count(c) AS countC...
3,Find all papers by a 'Daphne Koller',MATCH (a:author {name:'Daphne Koller'})-[:auth...
4,How many papers cite at least one other paper?,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...


In [ ]:
df2=pd.read_csv("/content/drive/MyDrive/cypher/medium_cypher.csv", header=None, names=['question', 'cypher'])
df2.head(5)

,question,cypher
0,How many papers were published each conference?,MATCH (p:paper)-[:paper_in_venue]->(c:conferen...
1,Which affiliations are most actively publishin...,MATCH (aff:affiliation)<-[:author_in_affiliati...
2,What are the key research domains of author 'Z...,MATCH (:author {name: 'Zhouchen Lin'})-[:autho...
3,Identify the papers in the field of Robotics b...,MATCH (:affiliation {name: 'University of Oxfo...
4,What are the top 5 main research areas in the ...,MATCH (p:paper)-[:paper_in_venue]->(:conferenc...


In [ ]:
df3=pd.read_csv("/content/drive/MyDrive/cypher/hard_cypher.csv", header=None, names=['question', 'cypher'])
df3.head(5)

,question,cypher
0,What's the most cited paper?,MATCH (p:paper)-[:paper_cite_paper]->(cited:pa...
1,Who are the authors of the paper 'Feature sele...,MATCH (p:paper {name: 'Feature selection for r...
2,Which papers cite the paper 'A sequential algo...,MATCH (p:paper {name: 'A sequential algorithm ...
3,In which conference was the paper 'Large scale...,MATCH (p:paper {name: 'Large scale semi-superv...
4,What are the affiliations of the authors of th...,MATCH (p:paper {name: 'A similarity-based prob...


In [ ]:
df = pd.concat([df1,df2,df3], axis=0)
df

,question,cypher
0,How many papers are in the database?,MATCH (p:paper) RETURN count(p) AS num_papers
1,Count the number of authors.,MATCH (a:Author) RETURN count(a) AS authorCount
2,How many conferences are listed?,MATCH (c:conference) RETURN count(c) AS countC...
3,Find all papers by a 'Daphne Koller',MATCH (a:author {name:'Daphne Koller'})-[:auth...
4,How many papers cite at least one other paper?,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...
...,...,...
114,Which papers on Maximum Likelihood have been p...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...
115,Identify recent popular papers in Augmented Re...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...
116,What are the trending research topics in Super...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...
117,List the latest papers in Bioinformatics with ...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...


## **Build Question-Answer**

In [ ]:
prefix="Generate cypher query for the question: "
suffix=" Only give cypher query, without any other words."
df['prompt'] = df['question'].apply(lambda x: prefix+x+suffix)
print(df.iloc[0]['prompt'])
df

Generate cypher query for the question: How many papers are in the database? Only give cypher query, without any other words.


,question,cypher,prompt
0,How many papers are in the database?,MATCH (p:paper) RETURN count(p) AS num_papers,Generate cypher query for the question: How ma...
1,Count the number of authors.,MATCH (a:Author) RETURN count(a) AS authorCount,Generate cypher query for the question: Count ...
2,How many conferences are listed?,MATCH (c:conference) RETURN count(c) AS countC...,Generate cypher query for the question: How ma...
3,Find all papers by a 'Daphne Koller',MATCH (a:author {name:'Daphne Koller'})-[:auth...,Generate cypher query for the question: Find a...
4,How many papers cite at least one other paper?,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,Generate cypher query for the question: How ma...
...,...,...,...
114,Which papers on Maximum Likelihood have been p...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,Generate cypher query for the question: Which ...
115,Identify recent popular papers in Augmented Re...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,Generate cypher query for the question: Identi...
116,What are the trending research topics in Super...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,Generate cypher query for the question: What a...
117,List the latest papers in Bioinformatics with ...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,Generate cypher query for the question: List t...


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, random_state=42)
len(train), len(test)

(153, 66)

In [ ]:
df_data = [{'input_text':train.iloc[i]['prompt'], 'output_text':train.iloc[i]['cypher']} for i in range(len(train))]
df_data[:2]

[{'input_text': "Generate cypher query for the question: Identify the collaboration network of 'Sebastian Thrun' within 'Carnegie Mellon University'. Only give cypher query, without any other words.",
  'output_text': "MATCH (a:author {name: 'Sebastian Thrun'})-[:author_write_paper]->(p:paper)\nMATCH (p)<-[:author_write_paper]-(coauthor:author)-[:author_in_affiliation]->(aff:affiliation {name:'Carnegie Mellon University'}) \nRETURN DISTINCT coauthor.name, aff.name"},
 {'input_text': "Generate cypher query for the question: Which paper in 'Parallel Computing' has authors from the most diverse affiliations? Only give cypher query, without any other words.",
  'output_text': "MATCH (p:paper)<-[:author_write_paper]-(a:author)-[:author_in_affiliation]->(aff:affiliation),\n(p)-[:paper_in_domain]->(:domain {name: 'Parallel Computing'})\nWITH p, COUNT(DISTINCT aff) AS numAffiliations\nRETURN p.name, numAffiliations\nORDER BY numAffiliations DESC\nLIMIT 1"}]

In [ ]:
pd.DataFrame(df_data)

,input_text,output_text
0,Generate cypher query for the question: Identi...,MATCH (a:author {name: 'Sebastian Thrun'})-[:a...
1,Generate cypher query for the question: Which ...,MATCH (p:paper)<-[:author_write_paper]-(a:auth...
2,Generate cypher query for the question: List a...,MATCH (aff:affiliation {name: 'University of O...
3,Generate cypher query for the question: What i...,MATCH (a:author {name: 'Daphne Koller'})-[:aut...
4,Generate cypher query for the question: How ma...,MATCH (aff:affiliation {name: 'Stanford Univer...
...,...,...
148,Generate cypher query for the question: Which ...,MATCH (a:author {name: 'Zhouchen Lin'})-[:auth...
149,Generate cypher query for the question: Find p...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...
150,Generate cypher query for the question: What a...,MATCH (a1:author)-[:author_write_paper]->(p:pa...
151,Generate cypher query for the question: What a...,MATCH (c:conference {name: 'SIGGRAPH'})<-[:pap...


## **Save to Jsonline**

In [ ]:
!pip install jsonlines -q
import jsonlines
with jsonlines.open("train_data.jsonl", "w") as f:
  for line in df_data:
    f.write(line)

In [ ]:
import json
with open("/content/drive/MyDrive/cypher/train_data.jsonl") as f:
    original = [json.loads(line) for line in f]
original[0]

{'input_text': "Generate cypher query for the question: Identify the collaboration network of 'Sebastian Thrun' within 'Carnegie Mellon University'. Only give cypher query, without any other words.",
 'output_text': "MATCH (a:author {name: 'Sebastian Thrun'})-[:author_write_paper]->(p:paper)\nMATCH (p)<-[:author_write_paper]-(coauthor:author)-[:author_in_affiliation]->(aff:affiliation {name:'Carnegie Mellon University'}) \nRETURN DISTINCT coauthor.name, aff.name"}

## **Load from Huggingface**

In [ ]:
!pip install -q datasets
import datasets
from datasets import load_dataset
from datasets import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00


In [ ]:
data_name = "jiuyuan/train_cypher"
training_data = load_dataset(data_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from datasets import Dataset
df_data = pd.DataFrame(training_data)
df_data

,input_text,output_text
0,Generate cypher query for the question: Identi...,MATCH (a:author {name: 'Sebastian Thrun'})-[:a...
1,Generate cypher query for the question: Which ...,MATCH (p:paper)<-[:author_write_paper]-(a:auth...
2,Generate cypher query for the question: List a...,MATCH (aff:affiliation {name: 'University of O...
3,Generate cypher query for the question: What i...,MATCH (a:author {name: 'Daphne Koller'})-[:aut...
4,Generate cypher query for the question: How ma...,MATCH (aff:affiliation {name: 'Stanford Univer...
...,...,...
148,Generate cypher query for the question: Which ...,MATCH (a:author {name: 'Zhouchen Lin'})-[:auth...
149,Generate cypher query for the question: Find p...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...
150,Generate cypher query for the question: What a...,MATCH (a1:author)-[:author_write_paper]->(p:pa...
151,Generate cypher query for the question: What a...,MATCH (c:conference {name: 'SIGGRAPH'})<-[:pap...


## **PaLM2 with VertexAI**

In [ ]:
from __future__ import annotations
from typing import Optional
from google.auth import default
from google.cloud import aiplatform
import pandas as pd
import vertexai
from vertexai.language_models import TextGenerationModel
from vertexai.preview.language_models import TuningEvaluationSpec


def tuning(
    project_id: str,
    location: str,
    model_display_name: str,
    training_data: pd.DataFrame | str,
    train_steps: int = 10,
) -> TextGenerationModel:
    """Tune a new model, based on a prompt-response data.

    "training_data" can be either the GCS URI of a file formatted in JSONL format
    (for example: training_data=f'gs://{bucket}/{filename}.jsonl'), or a pandas
    DataFrame. Each training example should be JSONL record with two keys, for
    example:
      {
        "input_text": <input prompt>,
        "output_text": <associated output>
      },
    or the pandas DataFame should contain two columns:
      ['input_text', 'output_text']
    with rows for each training example.

    Args:
      project_id: GCP Project ID, used to initialize vertexai
      location: GCP Region, used to initialize vertexai
      model_display_name: Customized Tuned LLM model name.
      training_data: GCS URI of jsonl file or pandas dataframe of training data
      train_steps: Number of training steps to use when tuning the model.
    """
    vertexai.init(project=project_id, location=location, credentials=credentials)
    model = TextGenerationModel.from_pretrained("text-bison@002")

    model.tune_model(
        training_data=training_data,
        # Optional:
        model_display_name=model_display_name,
        train_steps=train_steps,
        tuning_job_location="us-central1",
        tuned_model_location=location,
    )

    # print(model._job.status)
    return model


In [ ]:
!pip install google-cloud-aiplatform -q

In [ ]:
# !pip install -U google-cloud-aiplatform "shapely<2" -q

In [ ]:
from __future__ import annotations
from google.auth import default
import pandas as pd
import vertexai
from vertexai.preview.language_models import TextGenerationModel
# https://www.googleapis.com/auth/cloud-platform
credentials, _ = default(scopes=["/content/.config/application_default_credentials.json"])
project_id = 'alien-lattice-399217'
location = 'us-central1'
vertexai.init(project=project_id, location=location, credentials=credentials)

In [ ]:
!gcloud auth application-default set-quota-project alien-lattice-399217


Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "alien-lattice-399217" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [ ]:
!gcloud auth application-default login


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  Y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=Nnn2as6tOCTsL6EPICVIMYsbBm81oV&prompt=consent&access_type=offline&code_challenge=EgzTn

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
!gcloud config set project alien-lattice-399217

Updated property [core/project].


In [ ]:
# model = TextGenerationModel.from_pretrained("text-bison@001")


In [ ]:
# response=model.predict(prompt='hello, who are you?', max_output_tokens=25, temperature=0.2, top_k=2, top_p=0.5, )
# response

I am powered by PaLM 2, which stands for Pathways Language Model 2, a large language model from Google AI

## **Submit training job**

In [ ]:
project_id = 'alien-lattice-399217'
location = 'us-central1'
vertexai.init(project=project_id, location=location, credentials=credentials)
model_display_name='cypher500'
training_data = pd.DataFrame(df_data)
train_steps=500
model = tuning(project_id=project_id,
              location=location,
              model_display_name=model_display_name,
              training_data=training_data,
              train_steps=train_steps)


## **Fine tune Generations**

In [ ]:

df_test_all = pd.read_csv("llm_all_v2.csv")
df_test_all

,Unnamed: 0,prompt,palm_generations,question,cypher,test,fine_tuned_palm_generations,fine_tuned_llama_generations,fine_tuned_llama_cleaned_generations,llama_generations,llama_cleaned_generations,gpt_generations,gpt_fine_tuned_generations
0,0,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper) RETURN count(p),How many papers are in the database?,MATCH (p:paper) RETURN count(p) AS num_papers,1,MATCH (p:paper)\nRETURN COUNT(p),<s> Generate cypher query for the question: Ho...,MATCH (p:paper) RETURN count(p) AS num_papers,"To answer the question ""How many papers are ...","\nMATCH (p:Paper) RETURN p.id, COUNT(p)\n",MATCH (p:paper)\nRETURN count(p),MATCH (p:paper) RETURN count(p) AS num_papers
1,1,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:author) RETURN count(a),Count the number of authors.,MATCH (a:Author) RETURN count(a) AS authorCount,1,MATCH (a:author)\nRETURN COUNT(a) AS authorCount,<s> Generate cypher query for the question: Co...,MATCH (a:Author)\nRETURN count(a) AS authorCount,Sure! Here's a Cypher query to count the num...,\nMATCH (a:Author) RETURN COUNT(a)\n,MATCH (a:author)\nRETURN count(a) AS authorCount,MATCH (a:Author) RETURN count(a) AS authorCount
2,2,Given my Neo4j database schema:[<Record nodes=...,MATCH (c:conference) RETURN count(c),How many conferences are listed?,MATCH (c:conference) RETURN count(c) AS countC...,0,MATCH (c:conference)\nRETURN COUNT(c),<s> Generate cypher query for the question: Ho...,MATCH (c:conference) RETURN count(c) AS countC...,"Sure! To answer the question ""How many confere...",\nMATCH (c:Conference) RETURN c.name\n,MATCH (c:conference)\nRETURN COUNT(c) as confe...,MATCH (c:conference) RETURN count(c) AS countC...
3,3,Given my Neo4j database schema:[<Record nodes=...,"MATCH (a:author {name: ""Daphne Koller""})-[:aut...",Find all papers by a 'Daphne Koller',MATCH (a:author {name:'Daphne Koller'})-[:auth...,0,MATCH (a:author {name: 'Daphne Koller'})-[:au...,<s> Generate cypher query for the question: Fi...,MATCH (a:author {name:'Daphne Koller'})-[:auth...,"To find all papers authored by ""Daphne Koller""...","\nMATCH (a:Author {name: ""Daphne Koller""})-[:A...",MATCH (a:author {name: 'Daphne Koller'})-[:aut...,MATCH (a:author {name:'Daphne Koller'})-[:auth...
4,4,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[r:paper_cite_paper]->(p2:pape...,How many papers cite at least one other paper?,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,1,MATCH (p:paper)-[:paper_cite_paper]->(:paper)...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,"To answer the question ""How many papers cite a...",MATCH (p1:Paper)-[:CITES {num_cites: > 1}]- (p...,MATCH (p1:paper)-[:paper_cite_paper]->(p2:pape...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,214,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,Which papers on Maximum Likelihood have been p...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_venue]->(c:confere...,<s> Generate cypher query for the question: Wh...,MATCH (p:paper)-[:paper_in_venue]->(:conferenc...,To find the papers on Maximum Likelihood tha...,\nMATCH (p:Paper)-[:PRESENTED_AT]->(conf:Confe...,MATCH (p:paper)-[:paper_in_venue]->(c:conferen...,MATCH (p:paper)-[:paper_in_venue]->(c:conferen...
215,215,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain)...,Identify recent popular papers in Augmented Re...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_venue]->(c:conferen...,<s> Generate cypher query for the question: Id...,[],To identify recent popular papers in Augmented...,MATCH (paper:AugmentedReality)-[:PUBLISHED]->(...,MATCH (p:paper)-[:paper_in_venue]->(:conferenc...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...
216,216,Given my Neo4j database schema:[<Record nod

In [ ]:
def list_tuned_models(
    project_id: str,
    location: str,
) -> None:
    """List tuned models."""

    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@002")
    tuned_model_names = model.list_tuned_model_names()
    print(tuned_model_names)

    return tuned_model_names


list_tuned_models(project_id, location)

NameError: ignored

In [ ]:
model = TextGenerationModel.get_tuned_model('projects/330387137984/locations/us-central1/models/1678521048032608256')

In [ ]:
from tqdm.notebook import tqdm
import time
generations_fine_tuned = []
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = prompt = "Generate cypher query for the question: "+df_test_all.iloc[i]['question']+" Only give cypher query, without any other words."
  response=model.predict(prompt=prompt, max_output_tokens=100, temperature=0.2, top_k=2, top_p=0.5, )
  response_text = response.candidates[0]
  generations_fine_tuned.append(response_text)
  time.sleep(1)
generations_fine_tuned

generating...:   0%|          | 0/219 [00:00<?, ?it/s]

[ MATCH (p:paper)
 RETURN COUNT(p),
  MATCH (a:author)
 RETURN COUNT(a) AS authorCount,
  MATCH (c:conference)
 RETURN COUNT(c),
  MATCH (a:author {name: 'Daphne Koller'})-[:author_write_paper]->(p:paper)
 RETURN p.name,
  MATCH (p:paper)-[:paper_cite_paper]->(:paper)
 RETURN COUNT(DISTINCT p),
  MATCH (p:paper)<-[:paper_cite_paper]-(citingPaper:paper)
 RETURN COUNT(DISTINCT p),
  MATCH (p:paper)<-[:paper_cite_paper]-(citing:paper)
 WITH p, COUNT(citing) AS citations
 WHERE citations >= 10
 RETURN p.name, citations
 ORDER BY citations DESC,
  MATCH (d:domain)
 RETURN COUNT(d),
  MATCH (a:affiliation)
 RETURN a.name,
  MATCH (d:domain)
 RETURN d.name,
  MATCH (p:paper)-[:paper_in_domain]->(d:domain {name: 'Computer Science'})
 RETURN p.name,
  MATCH (p:paper)-[:paper_in_venue]->(c:conference {name: 'AAAI'}), (p)-[:paper_in_domain]->(:domain {name: 'Machine Learning'})
 RETURN p.name,
  MATCH (a:affiliation)
 RETURN COUNT(a) AS totalAffiliations,
  MATCH (a:author)-[:author_in_affiliatio

In [ ]:
df_test_all['fine_tuned_palm_generations'] = generations_fine_tuned

In [ ]:
df_test_all.to_csv("llm_all_v1.csv")

## **Generate for original model**

In [ ]:
model_original = TextGenerationModel.from_pretrained("text-bison@002")

In [ ]:
response=model_original.predict(prompt='hello, who are you?', max_output_tokens=25, temperature=0.2, top_k=2, top_p=0.5, )
response

I am powered by PaLM 2, which stands for Pathways Language Model 2, a large language model from Google AI

In [ ]:
from tqdm.notebook import tqdm
import time
generations = []
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = df_test_all.iloc[i]['prompt']
  response=model_original.predict(prompt=prompt, max_output_tokens=100, temperature=0.2, top_k=2, top_p=0.5, )
  response_text = response.candidates[0]
  generations.append(response_text)
  time.sleep(1)
generations

In [ ]:
df_test_all['palm2_generations'] = generations

In [ ]:
df_test_all['question'] = df['question']
df_test_all['cypher'] = df['cypher']
test_indices = list(df_test.index)
df_test_all['test'] = [1 if i in test_indices else 0 for i in range(len(df))]
df_test_all = df_test_all.reset_index()

In [ ]:
df_test_all=df_test_all.drop(columns=['index'])

In [ ]:
df_test_all.to_csv("Paml2_results.csv")

In [ ]:
df_test_all['palm2_generations'][55]

MATCH (p:paper)-[:paper_in_domain]->(d:domain {name: "Machine Learning"})-[:paper_in_venue]->(c:conference {name: "ACL"})-[:author_write_paper]->(a:author)-[:author_in_affiliation]->(f:affiliation {name: "Columbia University"}) RETURN count(p)

## **Generate for Fine Tuned Model**

In [ ]:
def list_tuned_models(
    project_id: str,
    location: str,
) -> None:
    """List tuned models."""

    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@002")
    tuned_model_names = model.list_tuned_model_names()
    print(tuned_model_names)

    return tuned_model_names


list_tuned_models(project_id, location)

['projects/330387137984/locations/us-central1/models/6933658863220555776']


['projects/330387137984/locations/us-central1/models/6933658863220555776']

In [ ]:

model = TextGenerationModel.get_tuned_model('projects/330387137984/locations/us-central1/models/6933658863220555776')

In [ ]:
df_test_all.iloc[54]['prompt']

"Given my Neo4j database schema:[<Record nodes=[<Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>] relationships=[<Relationship element_id='-7' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='author_write_paper' properties={'name': 'author_write_paper'

In [ ]:
prompt = df_test_all.iloc[54]['prompt']
response=model.predict(prompt=prompt, max_output_tokens=100, temperature=0.2, top_k=2, top_p=0.5, )
response.candidates[0]

 MATCH (c:conference {name: 'ACL'})<-[:paper_in_venue]-(p:paper)-[:paper_in_domain]->(d:domain)
RETURN d.name, COUNT(p) AS numPapers
ORDER BY numPapers DESC
LIMIT 5

In [ ]:
generations_ft = []
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = df_test_all.iloc[i]['prompt']
  response=model.predict(prompt=prompt, max_output_tokens=100, temperature=0.2, top_k=2, top_p=0.5, )
  response_text = response.candidates[0]
  generations_ft.append(response_text)
  time.sleep(1)
generations_ft

generating...:   0%|          | 0/219 [00:00<?, ?it/s]

[ MATCH (p:paper)
 RETURN COUNT(p),
  MATCH (a:author)
 RETURN COUNT(a),
  MATCH (c:conference)
 RETURN COUNT(c),
  MATCH (a:author {name: 'Daphne Koller'})-[:author_write_paper]->(p:paper)
 RETURN p.name,
  MATCH (p:paper)-[:paper_cite_paper]->(p2:paper)
 RETURN COUNT(DISTINCT p),
  MATCH (p:paper)<-[:paper_cite_paper]-(citing:paper)
 RETURN COUNT(DISTINCT p),
  MATCH (p:paper)<-[:paper_cite_paper]-(citing:paper)
 WITH p, COUNT(citing) AS numCiting
 WHERE numCiting >= 10
 RETURN p.name, numCiting
 ORDER BY numCiting DESC,
  MATCH (d:domain)
 RETURN COUNT(DISTINCT d),
  MATCH (a:affiliation)
 RETURN a.name,
  MATCH (d:domain)
 RETURN d.name,
  MATCH (p:paper)-[:paper_in_domain]->(d:domain {name: 'Computer Science'})
 RETURN p.name,
  MATCH (p:paper)-[:paper_in_venue]->(c:conference {name: 'AAAI'})<-[:author_write_paper]-(a:author)-[:author_in_affiliation]->(aff:affiliation)
 WHERE p.name CONTAINS 'Machine Learning'
 RETURN p.name, aff.name,
  MATCH (a:affiliation)
 RETURN COUNT(a),
  M

In [ ]:
df_test_all['palm2_fine_tuned_generations'] = generations_ft
df_test_all

,prompt,generations,question,cypher,test,fine_tuned_generations
0,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper) RETURN count(p),How many papers are in the database?,MATCH (p:paper) RETURN count(p) AS num_papers,1,MATCH (p:paper)\nRETURN COUNT(p)
1,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:author) RETURN count(a),Count the number of authors.,MATCH (a:Author) RETURN count(a) AS authorCount,1,MATCH (a:author)\nRETURN COUNT(a)
2,Given my Neo4j database schema:[<Record nodes=...,MATCH (c:conference) RETURN count(c),How many conferences are listed?,MATCH (c:conference) RETURN count(c) AS countC...,0,MATCH (c:conference)\nRETURN COUNT(c)
3,Given my Neo4j database schema:[<Record nodes=...,"MATCH (a:author {name: ""Daphne Koller""})-[:aut...",Find all papers by a 'Daphne Koller',MATCH (a:author {name:'Daphne Koller'})-[:auth...,0,MATCH (a:author {name: 'Daphne Koller'})-[:au...
4,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[r:paper_cite_paper]->(p2:pape...,How many papers cite at least one other paper?,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,1,MATCH (p:paper)-[:paper_cite_paper]->(p2:pape...
...,...,...,...,...,...,...
214,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,Which papers on Maximum Likelihood have been p...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_domain]->(d:domain...
215,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain)...,Identify recent popular papers in Augmented Re...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_venue]->(c:confere...
216,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain)...,What are the trending research topics in Super...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (d:domain {name: 'Supervised Learning'}...
217,Given my Neo4j database schema:[<Record nodes=...,```\nMATCH (p:paper)-[:paper_in_domain]->(d:do...,List the latest papers in Bioinformatics with ...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_domain]->(d:domain...


In [ ]:
df_test_all.to_csv("Paml2_fine_tuned_results.csv")